## Overview

On step 2, we segmented tissues from each slide, and tiled tissue region of slide to 256x256 images at level 1.

We select a pretrained ResNet-50 to extract features from tiles of each slide, outputting a 1024 dimentional vector.

**`256 x 256` images** ---ResNet50--->  **`1024` Vectors**

Note that image size of resnet input is `224 x 224`

## ResNet50 Structure


For extracting more low-level features from tiles, the pretrained model we used is a modified ResNet50 that only contains `conv1`, `conv2_x`, `conv3_x`, `conv4_x`, followed by a avgpool layer. Therefore, the data shape is transformed as below.

**`(W, H, 3)`** ---Modified ResNet50---> **`(W/16, H/16, 1024)`** ---AdaptiveAvgPool2d(1)---> **`(1024, )`**

## Extracing Features

Before feed images to model, ther are some important rounds to notice.

### Sampling patches

We firstly calculate energy for all patches of each slide, then sample N patches according to the image energy. Options for sampling methods:
- Random (random): random select N patches.
- Random By Energy (random-be): select top-M patches and then random select N patches.

### Determine the patch size

Firstly read the attributes of `patch_size` and `patch_level` restored in h5 file output at last step. Then, using the parameters as specified to calculate patch size as followed rules:

The parameter of `custom_downsample` is overruled by `target_patch_size`, used to determine the final patch size.

```python
with h5py.File(self.file_path, "r") as f:
	dset = f['coords']
	self.patch_level = f['coords'].attrs['patch_level']
	self.patch_size = f['coords'].attrsatch_size']
	self.length = len(dset)
	if target_patch_size > 0:
		self.target_patch_size = (target_patch_size, ) * 2
	elif custom_downsample > 1:
		self.target_patch_size = (self.patch_size // custom_downsample, ) * 2
	else:
		self.target_patch_size = None
```

### Read tiles from WSI and transform it

```python
# read tiles
img = self.wsi.read_region(coord, self.patch_level, (self.patch_size, self.patch_size)).convert('RGB')

# resize to target size
if self.target_patch_size is not None:
	img = img.resize(self.target_patch_size)
    
# transform the image
if not custom_transforms:
    mean = (0.485, 0.456, 0.406)
	std = (0.229, 0.224, 0.225)
	self.roi_transforms = transforms.Compose(
					[
					 transforms.ToTensor(),
					 transforms.Normalize(mean = mean, std = std)
					]
				)
else:
	self.roi_transforms = custom_transforms

img = self.roi_transforms(img).unsqueeze(0)
```

## Runing features extraction

runing below commands to extract features.

```bash
cd ../scripts
# calculate energy of images firstly
nohup ./3-Calculate-Patches-Energy.sh > 3-Calculate-Energy.log 2>&1 &
# extract features
nohup ./4-Extracting-Patches-Features.sh > 4-Extract-Feat.log 2>&1 &
```